This module builds upon data extracted in data scraping module. Here first we select stocks for which data exists 
for previous 10 years. From those stocks 150 stocks are randomly selected to create portfolio. After portfolio 
creation, I check for any missing data and impute such entries. Finally calculate correlation coefficient matrix
which can be passed on to ARIMA module for forecasting.

Cell 1 contains imports which will be required to run this module.

In [ ]:
#Cell 1 - Imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import os
import random

In [ ]:
#Cell 2 - Select stocks for which data is present since 2010

path = '/Users/gauravthapliyal/Desktop/Project Data/ticker_stock_data/'
stock_from2010 = []
for file in os.listdir(path):
    file_path = path + '/' + file
    date = pd.read_csv(file_path)['date']
    df=pd.DataFrame(data=date)
    if len(date)>0 and df['date'].iat[-1] <= '2010-01-01' :
        stock_from2010.append(file)

In [ ]:
#Cell 3 : Create a dictionary of Adjusted Close Price for all the stocks extracted in the previous cell.
stock_price_dict = {}

#run a loop for all the tickers for which data after 2010 is present.
#extract adjusted close price of stock and append in the dictionary.
for file in stock_from2010 :
    path = "/Users/gauravthapliyal/Desktop/Project Data/ticker_stock_data/" + file
    df_stock = pd.read_csv(path)
    df_stock = df_stock[df_stock.date >= '2010-01-01']
    pd.to_datetime(df_stock['date'], format='%Y-%m-%d')
    df_stock = df_stock.set_index(pd.DatetimeIndex(df_stock['date']))
    stock_price_dict[file.split(".")[0]] = df_stock['5. adjusted close']

#extract adjusted close value for combines ticker of SP500 firms and append that single column to dictionary
#SP500_index.csv contains stocks data for the combined representation ticker (GSPC) of SP500 firms
market_path = "/Users/gauravthapliyal/Desktop/Project Data/SP500_index.csv"
df = pd.read_csv(market_path)
df = df[df.Date >= '2010-01-01']
pd.to_datetime(df['Date'], format='%Y-%m-%d')
df = df.set_index(pd.DatetimeIndex(df['Date']))
stock_price_dict['SP500'] = df['Adj Close']

#save the dictionary in a csv
stock_price_df = pd.DataFrame(stock_price_dict)
data_dir = "/Users/gauravthapliyal/Desktop/Project Data/test.csv"
stock_price_df.to_csv(data_dir)
print(stock_price_df.head())

In [ ]:
#Cell 4 - Detecting anomalies in data

missing_ratio = []
missing_col = []

print("Values are not available for the following ticker")

#Checks if adjusted close price for any ticker is blank
#Counts the number of missing values for given index of a ticker

for column in stock_price_df.columns :
    miss_index = np.where(stock_price_df[column].isnull())[0]
    missing_col.append(column)
    missing_ratio.append(len(miss_index)/stock_price_df.shape[0] * 100)
    if miss_index > 0:
        print(column,miss_index)
        

Only a single ticker DISH is missing a single value. Since number of missing values is less than 5 for all the tickers, hence we will impute the value. For imputing we will use next day value and use it for the missing day. If more than 5 values missing, then uncomment deletion code and delete that ticket.

In [ ]:
#Cell 5 - Data Imputation

#Function definition to impute data.
def impute_data(column_name):
    index = stock_price_df.index.values[0]
    price_na_index = np.where(stock_price_df[column_name].isnull())[0]
    for i in price_na_index :
        stock_price_df[column_name][i] = stock_price_df[column_name][i-1]
        
for item in stock_price_df.columns :
    impute_data(item)

#corrupt_tickers = [] -- fill this array with tickers 
#for i in corrupt_tickers:
#    stock_price_df.remove(i)
    
print("Data is now clean. Proceeding to load data.")

stock_price_df.to_csv("/Users/gauravthapliyal/Desktop/Project Data/stock_pricefrom2010.csv",index_label='Date')

In [ ]:
#Cell 6 - Portfolio Creation

#Load dictionary of adjusted close prices
df = pd.read_csv("/Users/gauravthapliyal/Desktop/Project Data/stock_pricefrom2010.csv")
all_firms = list(df.columns.values[1:])
portfolio = list(stock_price_df.columns)
portfolio.pop()

#Create a portfolio of randomly selected 150 S&P500 firms
random.shuffle(all_firms)
portfolio = universe[:150].copy()
print(portfolio)

In [ ]:
#Cell 7 - Function for calculation rolling correlation matrix

def get_rolling_corr(item1,item2) :
    
    stock_price_df = pd.read_csv("/Users/gauravthapliyal/Desktop/Project Data/stock_pricefrom2010.csv")
    pd.to_datetime(stock_price_df['Date'], format='%Y-%m-%d')
    stock_price_df = stock_price_df.set_index(pd.DatetimeIndex(stock_price_df['Date']))
    
    df_pair = pd.concat([stock_price_df[item1], stock_price_df[item2]], axis=1)
    df_pair.columns = [item1,item2]
    df_corr = df_pair[item1].rolling(window=100).corr(df_pair[item2])
    return df_corr

In [ ]:
portfolio = ['FRT', 'AMD', 'MOS', 'DVN', 'RE', 'INTU', 'STE', 'JKHY', 'EXR', 'ZBRA', 'CMI', 'AMP', 'VNO', 'EMN', 'HSY', 'DAL', 'EQIX', 'ADP', 'PPL', 'LKQ', 'MS', 'EW', 'PPG', 'TSCO', 'MYL', 'XRX', 'VLO', 'JBHT', 'HPQ', 'FTI', 'FAST', 'AVGO', 'CVX', 'DGX', 'MGM', 'NVR', 'GE', 'LYV', 'REGN', 'AAL', 'PWR', 'RHI', 'EMR', 'ALL', 'HUM', 'CPRT', 'NLOK', 'CF', 'NVDA', 'HD', 'MRO', 'CMCSA', 'MET', 'MAR', 'DISCA', 'SLG', 'URI', 'RL', 'FLS', 'BAC', 'NEE', 'CAT', 'DG', 'PCAR', 'DLR', 'NI', 'GILD', 'O', 'TEL', 'ESS', 'DLTR', 'GS', 'ALK', 'KSS', 'NOV', 'UNP', 'ES', 'HAL', 'GLW', 'TAP', 'SPGI', 'CVS', 'IPG', 'UNH', 'WYNN', 'PXD', 'IEX', 'DIS', 'ROP', 'MKTX', 'NEM', 'LDOS', 'HOG', 'CNC', 'TJX', 'AMAT', 'MDT', 'PVH', 'AKAM', 'INCY', 'TROW', 'HWM', 'ULTA', 'KR', 'AAPL', 'PNC', 'BDX', 'ADM', 'AJG', 'BXP', 'SRE', 'VZ', 'NTRS', 'HON', 'BEN', 'TRV', 'HAS', 'CCI', 'FIS', 'FFIV', 'KEY', 'CTSH', 'IDXX', 'DRE', 'DISH', 'MSI', 'ADS', 'CDNS', 'ADI', 'AOS', 'KLAC', 'HRB', 'KMX', 'ISRG', 'EL', 'GOOGL', 'ARE', 'PEP', 'CTL', 'EXC', 'TGT', 'HIG', 'ROL', 'COO', 'CI', 'SBAC', 'BKNG', 'GWW', 'WY', 'RJF']
print(len(portfolio))

In [ ]:
#Cell 8 Calculate correlation cpefficient matrix and saving it in csv form

#create a list of indices
index_list = []
for _ in range(100):
    indices = []
    for k in range(_, 2420,100):
        indices.append(k)
    index_list.append(indices)

#calculate rolling correlation
data_matrix = []
count = 0
for i in range(150):
    for j in range(149-i):
        a = portfolio[i]
        b = portfolio[149-j]
        file_name = a + '_' + b
            
        corr_series = get_rolling_corr(a, b)[99:]
        for _ in range(100):
            corr_strided = list(corr_series[index_list[_]][:24]).copy()
            data_matrix.append(corr_strided)
            count+=1
            if count % 1000 == 0 :
                print(str(count)+' items extracted and transformed')

#calculate coefficient correlation matrix and save it in csv form
data_matrix = np.transpose(data_matrix)
data_dictionary = {}
for i in range(len(data_matrix)):
    data_dictionary[str(i)] = data_matrix[i]
data_df = pd.DataFrame(data_dictionary)
data_df.to_csv('/Users/gauravthapliyal/Desktop/Project Data/stock_correlation_prediction/Correlation_Matrix.csv')